In [6]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

NUM_LANDMARKS = 63
model = load_model('model_numbers.h5')
GESTURES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append([landmark.x, landmark.y, landmark.z])

            landmarks = np.array(landmarks).flatten()
            landmarks = landmarks / np.max(landmarks)

            if landmarks.shape[0] == NUM_LANDMARKS:
                landmarks_input = landmarks.reshape(1, NUM_LANDMARKS, 1)
                predictions = model.predict(landmarks_input)
                predicted_index = np.argmax(predictions)
                predicted_gesture = GESTURES[predicted_index]
                cv2.putText(frame, f'Predicted: {predicted_gesture}', (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Hand Gesture Prediction', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 21ms/step
